In [76]:
import os
os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

import sys
sys.path.append("..")

from datetime import datetime

import numpy as np
import pandas as pd

import pyarrow
import pyarrow.compute as pc

import brasa

In [118]:
df = brasa.get_dataset("b3-company-details").scanner(columns=["issuingCompany", "refdate"]).to_table().to_pandas()
df = df.groupby(["issuingCompany", "refdate"], sort=True).last().reset_index()

comp_det = brasa.get_dataset("b3-company-details")\
    .to_table()\
    .to_pandas()

comp_det = pd.merge(df, comp_det, on=["issuingCompany", "refdate"], how="inner")

comp_det["issuingCompany"] = comp_det["issuingCompany"].astype(str).str.strip()
comp_det["companyName"] = comp_det["companyName"].astype(str).str.strip()
comp_det["tradingName"] = comp_det["tradingName"].astype(str).str.strip()
comp_det["cnpj"] = comp_det["cnpj"].astype(str).str.strip()
comp_det["industryClassification"] = comp_det["industryClassification"].astype(str).str.strip()
comp_det["industryClassificationEng"] = comp_det["industryClassificationEng"].astype(str).str.strip()
comp_det["activity"] = comp_det["activity"].astype(str).str.strip()
comp_det["website"] = comp_det["website"].astype(str).str.strip()
comp_det["hasQuotation"] = comp_det["hasQuotation"].astype(bool)
comp_det["status"] = comp_det["status"].astype(str).str.strip()
comp_det["marketIndicator"] = comp_det["marketIndicator"].astype("int64")
comp_det["market"] = comp_det["market"].astype(str).str.strip()
comp_det["institutionCommon"] = comp_det["institutionCommon"].astype(str).str.strip()
comp_det["institutionPreferred"] = comp_det["institutionPreferred"].astype(str).str.strip()
comp_det["code"] = comp_det["code"].astype(str).str.strip()
comp_det["codeCVM"] = comp_det["codeCVM"].astype("int64")
comp_det["lastDate"] = pd.to_datetime(comp_det["lastDate"], dayfirst=True)
comp_det["hasEmissions"] = comp_det["hasEmissions"].astype(bool)
comp_det["hasBDR"] = comp_det["hasBDR"].astype(bool)
comp_det["typeBDR"] = comp_det["typeBDR"].astype(str).str.strip()
comp_det["describleCategoryBVMF"] = comp_det["describleCategoryBVMF"].astype(str).str.strip()
comp_det["isin"] = comp_det["isin"].astype(str).str.strip()
comp_det["refdate"] = pd.to_datetime(comp_det["refdate"])

comp_det = comp_det.rename(columns={
    "issuingCompany": "asset_name",
    "companyName": "company_name",
    "tradingName": "trading_name",
    "industryClassification": "industry_classification",
    "industryClassificationEng": "industry_classification_eng",
    "hasQuotation": "has_quotation",
    "marketIndicator": "market_indicator",
    "market": "market",
    "institutionCommon": "institution_common",
    "institutionPreferred": "institution_preferred",
    "codeCVM": "code_cvm",
    "lastDate": "last_date",
    "hasEmissions": "has_emissions",
    "hasBDR": "has_bdr",
    "typeBDR": "type_bdr",
    "describleCategoryBVMF": "describle_category_bvmf",
    "code": "symbol",
})

industry_sectors = comp_det["industry_classification"].str\
    .replace(r" +/ +", "/", regex=True).str\
    .split("/", expand=True)\
    .rename(columns={0: "sector", 1: "subsector", 2: "segment"})
comp_det[["sector", "subsector", "segment"]] = industry_sectors

comp_det = comp_det.replace({"None": pd.NA})
comp_det

,asset_name,refdate,company_name,trading_name,cnpj,industry_classification,industry_classification_eng,activity,website,has_quotation,...,code_cvm,last_date,has_emissions,has_bdr,type_bdr,describle_category_bvmf,isin,sector,subsector,segment
0,AALR,2023-12-21,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,ALLIAR,42771949000135,Saúde / Serv.Méd.Hospit..Análises e Diagnóstic...,<NA>,Prestação de serviços de medicina diagnóstica,http://ri.alliar.com/,False,...,24058,2023-12-20 23:09:21,False,False,<NA>,<NA>,BRAALRACNOR6,Saúde,Serv.Méd.Hospit..Análises e Diagnósticos,Serv.Méd.Hospit..Análises e Diagnósticos
1,ABCB,2023-12-21,BCO ABC BRASIL S.A.,ABC BRASIL,28195667000106,Financeiro / Intermediários Financeiros / Bancos,<NA>,O Banco tem por objeto a prática de operações ...,www.abcbrasil.com.br,False,...,20958,2023-12-20 23:14:59,False,False,<NA>,<NA>,BRABCBACNPR4,Financeiro,Intermediários Financeiros,Bancos
2,ABEV,2023-12-21,AMBEV S.A.,AMBEV S/A,07526557000100,Consumo não Cíclico / Bebidas / Cervejas e Ref...,<NA>,Fabricação e distribuição de cervejas. refrige...,http://ri.ambev.com.br,False,...,23264,2023-12-20 23:10:10,False,False,<NA>,<NA>,BRABEVACNOR1,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
3,AERI,2023-12-21,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,AERIS,12528708000107,Bens Industriais / Máquinas e Equipamentos / M...,<NA>,Exploração de negócio de construção e comercia...,,False,...,25283,2023-12-20 23:07:37,False,False,<NA>,<NA>,BRAERIACNOR4,Bens Industriais,Máquinas e Equipamentos,Máq. e Equip. Industriais
4,AESB,2023-12-21,AES BRASIL ENERGIA S.A.,AES BRASIL,37663076000107,Utilidade Pública / Energia Elétrica / Energia...,<NA>,HOLDING DE INSTITUIÇÕES NÃO FINANCEIRAS,http://ri.aesbrasil.com.br,False,...,25640,2023-12-20 23:07:40,False,False,<NA>,<NA>,BRAESBACNOR7,Utilidade Pública,Energia Elétrica,Energia Elétrica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,WNBR,2023-12-21,W2W E-COMMERCE DE VINHOS S.A.,WINE,09813204000116,Não Classificados / Não Classificado / Não Cla...,<NA>,Plataforma digital de compra e venda de vinhos...,,False,...,25852,2023-12-21 00:02:35,False,False,<NA>,<NA>,<NA>,Não Classificados,Não Classificado,Não Classificados
608,YBRA,2023-12-21,YBYRÁ CAPITAL S.A.,YBYRA S/A,02217319000107,Outros / Outros / Outros,<NA>,Holding - participação em outras sociedades,https://ybyracapital.com.br/ri/,False,...,16705,2023-12-21 00:03:44,False,False,<NA>,<NA>,BRYBRAACNOR7,Outros,Outros,Outros
609,YBRA,2023-12-21,YBYRÁ CAPITAL S.A.,YBYRA S/A,02217319000107,Outros / Outros / Outros,<NA>,Holding - participação em outras sociedades,https://ybyracapital.com.br/ri/,False,...,16705,2023-12-21 00:03:44,False,False,<NA>,<NA>,BRYBRAACNPR4,Outros,Outros,Outros
610,YDUQ,2023-12-21,YDUQS PARTICIPACOES S.A.,YDUQS PART,08807432000110,Consumo Cíclico / Diversos / Serviços Educacio...,<NA>,Desenvolve principalmente nas áreas de educaçã...,www.estacioparticipacoes.com.br,False,...,21016,2023-12-21 00:03:45,False,False,<NA>,<NA>,BRYDUQACNOR3,Consumo Cíclico,Diversos,Serviços Educacionais


In [124]:
comp_det[["symbol", "sector", "subsector", "segment"]].dropna(axis=0)

,symbol,sector,subsector,segment
0,AALR3,Saúde,Serv.Méd.Hospit..Análises e Diagnósticos,Serv.Méd.Hospit..Análises e Diagnósticos
1,ABCB4,Financeiro,Intermediários Financeiros,Bancos
2,ABEV3,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
3,AERI3,Bens Industriais,Máquinas e Equipamentos,Máq. e Equip. Industriais
4,AESB3,Utilidade Pública,Energia Elétrica,Energia Elétrica
...,...,...,...,...
606,WLMM4,Bens Industriais,Comércio,Material de Transporte
608,YBRA3,Outros,Outros,Outros
609,YBRA4,Outros,Outros,Outros
610,YDUQ3,Consumo Cíclico,Diversos,Serviços Educacionais


In [113]:
df = brasa.get_dataset("b3-company-info-report").scanner(columns=["code", "refdate"]).to_table().to_pandas()
df = df.groupby(["code", "refdate"], sort=True).last().reset_index()

comp_info = brasa.get_dataset("b3-company-info-report")\
    .to_table()\
    .to_pandas()

comp_info = pd.merge(df, comp_info, on=["code", "refdate"], how="inner")

comp_info["stockCapital"] = comp_info["stockCapital"].astype("float64")
comp_info["commonSharesForm"] = comp_info["commonSharesForm"].astype(str).str.strip()
comp_info["preferredSharesForm"] = comp_info["preferredSharesForm"].astype(str).str.strip()
comp_info["hasCommom"] = comp_info["hasCommom"].astype(bool)
comp_info["hasPreferred"] = comp_info["hasPreferred"].astype(bool)
comp_info["roundLot"] = comp_info["roundLot"].astype("int64")
comp_info["tradingName"] = comp_info["tradingName"].astype(str).str.strip()
comp_info["numberCommonShares"] = comp_info["numberCommonShares"].astype("int64")
comp_info["numberPreferredShares"] = comp_info["numberPreferredShares"].astype("int64")
comp_info["totalNumberShares"] = comp_info["totalNumberShares"].astype("int64")
comp_info["code"] = comp_info["code"].astype(str).str.strip()
comp_info["codeCVM"] = comp_info["codeCVM"].astype("int64")
comp_info["segment"] = comp_info["segment"].astype(str).str.strip()
comp_info["refdate"] = pd.to_datetime(comp_info["refdate"])

comp_info = comp_info\
    .rename(columns={
        "tradingName": "trading_name",
        "code": "asset_name",
        "codeCVM": "code_cvm",
        "quotedPerSharSince": "quoted_per_shar_since",
        "commonSharesForm": "common_shares_form",
        "preferredSharesForm": "preferred_shares_form",
        "hasCommom": "has_common",
        "hasPreferred": "has_preferred",
        "roundLot": "round_lot",
        "stockCapital": "stock_capital",
        "numberCommonShares": "number_common_shares",
        "numberPreferredShares": "number_preferred_shares",
        "totalNumberShares": "total_number_shares",
    })

comp_info = comp_info.replace({"None": pd.NA})

comp_info

,asset_name,refdate,stock_capital,quoted_per_shar_since,common_shares_form,preferred_shares_form,has_common,has_preferred,round_lot,trading_name,number_common_shares,number_preferred_shares,total_number_shares,code_cvm,segment
0,A6OP,2023-12-20,1.000000e+00,2023-04-25,Escritural,,True,False,100,ACESSOPAR,1,0,1,0,BOLSA
1,AALR,2023-12-20,6.353720e+08,2016-10-14,Escritural,Escritural,True,True,100,ALLIAR,118292816,0,118292816,24058,NM
2,ABCB,2023-12-20,4.472131e+09,2007-07-25,Escritural,Escritural,True,True,100,ABC BRASIL,119010510,117925586,236936096,20958,N2
3,ABEV,2023-12-20,5.817793e+10,2013-11-01,Escritural,,True,False,100,AMBEV S/A,15753833284,0,15753833284,23264,BOLSA
4,AERI,2023-12-20,9.119750e+08,2020-10-20,Escritural,,True,False,100,AERIS,1242403933,0,1242403933,25283,NM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,WNBR,2023-12-20,1.052593e+08,2020-10-08,Escritural,,True,False,100,WINE,86540918,0,86540918,25852,MB
519,YBRA,2023-12-20,4.665090e+09,2023-06-28,Escritural,Escritural,True,True,100,YBYRA S/A,116574000,116574000,233148000,16705,BOLSA
520,YDUQ,2023-12-20,1.139887e+09,2007-07-30,Escritural,Escritural,True,True,100,YDUQS PART,309088851,0,309088851,21016,NM
521,YOUC,2023-12-20,7.534782e+07,2020-06-25,Escritural,,True,False,100,YOU INC,32317787,0,32317787,0,BOLSA


In [114]:
cd0 = comp_det[["asset_name", "company_name", "trading_name", "cnpj", "code_cvm", "industry_classification", "activity", "website", "market_indicator", "market", "refdate"]]\
    .query("code_cvm != 0")
cd0 = cd0.drop_duplicates()

ci0 = comp_info[[
    "trading_name", "asset_name", "code_cvm", "segment", "has_common", "has_preferred", "quoted_per_shar_since",
    "round_lot", "stock_capital", "number_common_shares", "number_preferred_shares", "total_number_shares", "refdate"]]\
    .query("code_cvm != 0")

companies_properties = pd.merge(cd0, ci0, on=["asset_name", "code_cvm", "trading_name"], how="outer")
companies_properties

,asset_name,company_name,trading_name,cnpj,code_cvm,industry_classification,activity,website,market_indicator,market,...,segment,has_common,has_preferred,quoted_per_shar_since,round_lot,stock_capital,number_common_shares,number_preferred_shares,total_number_shares,refdate_y
0,AALR,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,ALLIAR,42771949000135,24058,Saúde / Serv.Méd.Hospit..Análises e Diagnóstic...,Prestação de serviços de medicina diagnóstica,http://ri.alliar.com/,18.0,NOVO MERCADO,...,NM,True,True,2016-10-14,100,6.353720e+08,118292816,0,118292816,2023-12-20
1,ABCB,BCO ABC BRASIL S.A.,ABC BRASIL,28195667000106,20958,Financeiro / Intermediários Financeiros / Bancos,O Banco tem por objeto a prática de operações ...,www.abcbrasil.com.br,17.0,BOVESPA NIVEL 2,...,N2,True,True,2007-07-25,100,4.472131e+09,119010510,117925586,236936096,2023-12-20
2,ABEV,AMBEV S.A.,AMBEV S/A,07526557000100,23264,Consumo não Cíclico / Bebidas / Cervejas e Ref...,Fabricação e distribuição de cervejas. refrige...,http://ri.ambev.com.br,1.0,BOLSA,...,BOLSA,True,False,2013-11-01,100,5.817793e+10,15753833284,0,15753833284,2023-12-20
3,AERI,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,AERIS,12528708000107,25283,Bens Industriais / Máquinas e Equipamentos / M...,Exploração de negócio de construção e comercia...,,18.0,NOVO MERCADO,...,NM,True,False,2020-10-20,100,9.119750e+08,1242403933,0,1242403933,2023-12-20
4,AESB,AES BRASIL ENERGIA S.A.,AES BRASIL,37663076000107,25640,Utilidade Pública / Energia Elétrica / Energia...,HOLDING DE INSTITUIÇÕES NÃO FINANCEIRAS,http://ri.aesbrasil.com.br,18.0,NOVO MERCADO,...,NM,True,False,2021-03-25,100,2.196958e+09,601927311,0,601927311,2023-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,OPDL,NaN,DALETH PART,NaN,16950,NaN,NaN,NaN,NaN,NaN,...,MB,True,False,1998-11-11,100,9.970495e+04,91042901,0,91042901,2023-12-20
421,ORNA,NaN,ORNATO,NaN,504122,NaN,NaN,NaN,NaN,NaN,...,MB,True,True,2011-04-08,1000,1.000000e+00,0,1,1,2023-12-20
422,RNPT,NaN,RENNER PART,NaN,17752,NaN,NaN,NaN,NaN,NaN,...,BOLSA,True,True,2006-05-19,100,5.600000e+07,295306,480960,776266,2023-12-20
423,SPCI,NaN,CIMBRASIL,NaN,16217,NaN,NaN,NaN,NaN,NaN,...,BOLSA,True,True,1997-04-25,1000,6.840938e+07,164299622,60336528,224636150,2023-12-20


In [117]:
companies_symbols = comp_det[["symbol", "asset_name", "trading_name", "company_name", "code_cvm", "isin"]].copy()
companies_symbols["stock_type"] = companies_symbols["isin"].str[9:11]\
    .map({"PR": "PN", "OR": "ON", "PA": "PNA", "PB": "PNB", "M1": "UNT"})

symbol_info = comp_info[["trading_name", "asset_name", "code_cvm", "segment"]].copy()

symbols_properties = pd.merge(companies_symbols, symbol_info, on=("asset_name", "code_cvm", "trading_name"))
symbols_properties = symbols_properties[~symbols_properties["symbol"].isna()]
symbols_properties

,symbol,asset_name,trading_name,company_name,code_cvm,isin,stock_type,segment
0,AALR3,AALR,ALLIAR,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,24058,BRAALRACNOR6,ON,NM
1,ABCB4,ABCB,ABC BRASIL,BCO ABC BRASIL S.A.,20958,BRABCBACNPR4,PN,N2
2,ABEV3,ABEV,AMBEV S/A,AMBEV S.A.,23264,BRABEVACNOR1,ON,BOLSA
3,AERI3,AERI,AERIS,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,25283,BRAERIACNOR4,ON,NM
4,AESB3,AESB,AES BRASIL,AES BRASIL ENERGIA S.A.,25640,BRAESBACNOR7,ON,NM
...,...,...,...,...,...,...,...,...
606,WLMM4,WLMM,WLM IND COM,WLM PART. E COMÉRCIO DE MÁQUINAS E VEÍCULOS S.A.,11070,BRWLMMACNPR3,PN,BOLSA
608,YBRA3,YBRA,YBYRA S/A,YBYRÁ CAPITAL S.A.,16705,BRYBRAACNOR7,ON,BOLSA
609,YBRA4,YBRA,YBYRA S/A,YBYRÁ CAPITAL S.A.,16705,BRYBRAACNPR4,PN,BOLSA
610,YDUQ3,YDUQ,YDUQS PART,YDUQS PARTICIPACOES S.A.,21016,BRYDUQACNOR3,ON,NM


In [37]:
cd = brasa.get_dataset("b3-cash-dividends").to_table().to_pandas()
cd.head()

,typeStock,dateApproval,valueCash,ratio,corporateAction,lastDatePriorEx,dateClosingPricePriorExDate,closingPricePriorExDate,quotedPerShares,corporateActionPrice,lastDateTimePriorEx,tradingName,refdate
0,ON,2023-02-09,0.115164,1,DIVIDENDO,2023-02-14,2023-02-14,12.08,1,0.953342,2023-02-14,3TENTOS,2023-12-22
1,ON,2022-04-27,0.015304,1,DIVIDENDO,2022-04-27,2022-04-27,8.87,1,0.172542,2022-04-27,3TENTOS,2023-12-22
2,ON,2014-12-23,0.333000,1,JRS CAP PROPRIO,2014-12-30,NaT,0.00,1,NaN,2014-12-30,ABC BRASIL,2023-12-22
3,ON,2014-06-27,0.326000,1,JRS CAP PROPRIO,2014-06-30,NaT,0.00,1,NaN,2014-06-30,ABC BRASIL,2023-12-22
4,ON,2013-12-26,0.303000,1,JRS CAP PROPRIO,2013-12-30,NaT,0.00,1,NaN,2013-12-30,ABC BRASIL,2023-12-22


In [69]:
cd_ = cd[["typeStock", "tradingName", "dateApproval", "lastDatePriorEx", "valueCash", "ratio", "corporateAction"]]\
    .rename(columns={
        "dateApproval": "approved_on",
        "lastDatePriorEx": "last_date_prior_ex",
        "valueCash": "value_cash",
        "corporateAction": "corporate_action_label",
        "typeStock": "stock_type",
        "tradingName": "trading_name",
    })
cd_["payment_date"] = pd.NaT
cd_ = pd.merge(cd_, symbols_properties[["stock_type", "trading_name", "symbol"]], on=["stock_type", "trading_name"]).drop(columns=["stock_type", "trading_name"])
cd_

,approved_on,last_date_prior_ex,value_cash,ratio,corporate_action_label,payment_date,symbol
0,2023-02-09,2023-02-14,0.115164,1,DIVIDENDO,NaT,TTEN3
1,2022-04-27,2022-04-27,0.015304,1,DIVIDENDO,NaT,TTEN3
2,2023-06-26,2023-06-30,0.836000,1,JRS CAP PROPRIO,NaT,ABCB4
3,2022-12-26,2022-12-29,0.370000,1,JRS CAP PROPRIO,NaT,ABCB4
4,2022-09-26,2022-09-30,0.362000,1,JRS CAP PROPRIO,NaT,ABCB4
...,...,...,...,...,...,...,...
18213,2010-04-19,2010-04-19,0.388400,1,DIVIDENDO,NaT,YDUQ3
18214,2009-04-28,2009-04-28,0.227300,1,DIVIDENDO,NaT,YDUQ3
18215,2008-04-25,2008-04-25,0.057900,1,DIVIDENDO,NaT,YDUQ3
18216,2020-02-19,2020-03-02,0.050815,1,JRS CAP PROPRIO,NaT,ZAMP3


In [70]:
cd1 = brasa.get_dataset("b3-company-info-cash_dividends").to_table().to_pandas()
cd1.head()

,assetIssued,paymentDate,rate,relatedTo,approvedOn,isinCode,label,lastDatePrior,remarks,refdate
0,BRAALRACNOR6,2020-12-18,0.087298,ANUAL/2019,2020-04-27,BRAALRACNOR6,DIVIDENDO,2020-04-27,,2023-12-20
1,BRABCBACNPR4,2023-08-16,0.836000,1º Semestre/2023,2023-06-26,BRABCBACNPR4,JRS CAP PROPRIO,2023-06-30,,2023-12-20
2,BRABCBACNPR4,2023-01-13,0.370000,4º Trimestre/2022,2022-12-26,BRABCBACNPR4,JRS CAP PROPRIO,2022-12-29,,2023-12-20
3,BRABEVACNOR1,2023-12-28,0.730200,4º Trimestre/2023,2023-12-12,BRABEVACNOR1,JRS CAP PROPRIO,2023-12-19,,2023-12-20
4,BRABEVACNOR1,2021-12-30,0.133400,Anual/2021,2021-12-09,BRABEVACNOR1,DIVIDENDO,2021-12-17,,2023-12-20


In [71]:
cd1_ = cd1[["isinCode", "paymentDate", "approvedOn", "lastDatePrior", "rate", "label"]]\
    .rename(columns={
        "isinCode": "isin",
        "paymentDate": "payment_date",
        "approvedOn": "approved_on",
        "lastDatePrior": "last_date_prior_ex",
        "label": "corporate_action_label",
        "rate": "value_cash",
    })
cd1_ = pd.merge(cd1_, symbols_properties[["isin", "symbol"]], on="isin").drop(columns="isin")
cd1_["ratio"] = pd.NA
cd1_

,payment_date,approved_on,last_date_prior_ex,value_cash,corporate_action_label,symbol,ratio
0,2020-12-18,2020-04-27,2020-04-27,0.087298,DIVIDENDO,AALR3,NaN
1,2023-08-16,2023-06-26,2023-06-30,0.836000,JRS CAP PROPRIO,ABCB4,NaN
2,2023-01-13,2022-12-26,2022-12-29,0.370000,JRS CAP PROPRIO,ABCB4,NaN
3,2023-12-28,2023-12-12,2023-12-19,0.730200,JRS CAP PROPRIO,ABEV3,NaN
4,2021-12-30,2021-12-09,2021-12-17,0.133400,DIVIDENDO,ABEV3,NaN
...,...,...,...,...,...,...,...
1593,2023-08-25,2023-06-28,2023-08-08,0.229173,JRS CAP PROPRIO,WLMM4,NaN
1594,2023-06-27,2023-03-20,2023-04-28,0.169400,DIVIDENDO,WLMM4,NaN
1595,2023-06-27,2023-03-20,2023-04-28,0.269500,DIVIDENDO,WLMM4,NaN
1596,2023-02-27,2022-12-29,2023-01-06,0.286466,JRS CAP PROPRIO,WLMM4,NaN


In [73]:
cash_dividends = pd.concat([cd_, cd1_])

In [75]:
cash_dividends.query("symbol == 'MGLU3'").sort_values("approved_on")

,approved_on,last_date_prior_ex,value_cash,ratio,corporate_action_label,payment_date,symbol
12044,2012-04-30,2012-04-30,0.014857,1,DIVIDENDO,NaT,MGLU3
12043,2014-01-30,2014-02-20,0.065683,1,JRS CAP PROPRIO,NaT,MGLU3
12042,2014-04-17,2014-04-22,0.107359,1,DIVIDENDO,NaT,MGLU3
12041,2014-12-30,2015-01-14,0.078168,1,JRS CAP PROPRIO,NaT,MGLU3
12040,2015-04-27,2015-04-27,0.109472,1,DIVIDENDO,NaT,MGLU3
12039,2017-04-20,2017-04-25,1.017261,1,DIVIDENDO,NaT,MGLU3
12038,2017-12-13,2017-12-19,0.396103,1,JRS CAP PROPRIO,NaT,MGLU3
12037,2018-04-13,2018-04-16,0.264465,1,DIVIDENDO,NaT,MGLU3
12036,2018-12-21,2018-12-28,0.592705,1,JRS CAP PROPRIO,NaT,MGLU3
12035,2019-04-12,2019-04-15,0.370260,1,DIVIDENDO,NaT,MGLU3


In [76]:
sd0 = brasa.get_dataset("b3-company-info-stock_dividends").to_table().to_pandas()
sd0.head()

,assetIssued,factor,approvedOn,isinCode,label,lastDatePrior,remarks,refdate
0,BRAGROACNOR7,9900.0,2007-10-31,BRAGROACNOR7,DESDOBRAMENTO,2007-10-31,,2023-12-20
1,,100.0,2019-04-23,BRAHEBR02PA6,RESG TOTAL RV,2019-04-23,,2023-12-20
2,,100.0,2019-04-23,BRAHEBR02PB4,RESG TOTAL RV,2019-04-23,,2023-12-20
3,,100.0,2019-04-23,BRAHEBR05OR6,RESG TOTAL RV,2019-04-23,,2023-12-20
4,BRALPAACNOR0,25.0,2019-03-20,BRALPAACNOR0,BONIFICACAO,2019-03-20,,2023-12-20


In [82]:
sd0_ = sd0[["isinCode", "label", "lastDatePrior", "approvedOn", "factor"]]\
    .rename(columns={
        "isinCode": "isin",
        "label": "corporate_action_label",
        "lastDatePrior": "last_date_prior_ex",
        "approvedOn": "approved_on",
    })
sd0_ = pd.merge(sd0_, symbols_properties[["isin", "symbol"]], on="isin").drop(columns="isin")
sd0_

,corporate_action_label,last_date_prior_ex,approved_on,factor,symbol
0,DESDOBRAMENTO,2007-10-31,2007-10-31,9900.000,AGRO3
1,BONIFICACAO,2019-03-20,2019-03-20,25.000,ALPA3
2,DESDOBRAMENTO,2010-02-24,2010-02-24,1900.000,ALPA3
3,GRUPAMENTO,2005-06-08,2005-04-01,0.010,ALPA3
4,BONIFICACAO,2019-03-20,2019-03-20,25.000,ALPA4
...,...,...,...,...,...
722,DESDOBRAMENTO,2011-01-05,2011-01-05,300.000,VULC3
723,BONIFICACAO,1986-02-27,1986-02-27,200.000,VULC3
724,DESDOBRAMENTO,2021-04-27,2021-04-27,100.000,WEGE3
725,BONIFICACAO,2018-04-24,2018-04-24,30.000,WEGE3


In [83]:
sd0_.query("symbol == "MGLU3"").sort_values("approved_on")

,corporate_action_label,last_date_prior_ex,approved_on,factor,symbol
459,GRUPAMENTO,2015-09-30,2015-09-30,0.125,MGLU3
458,DESDOBRAMENTO,2020-10-13,2020-10-07,300.000,MGLU3


In [79]:
su0 = brasa.get_dataset("b3-company-info-subscriptions").to_table().to_pandas()
su0.head()

,assetIssued,percentage,priceUnit,tradingPeriod,subscriptionDate,approvedOn,isinCode,label,lastDatePrior,remarks,refdate
0,BRABCBACNPR4,4.905471,14.485865,03/07/2023 a 31/07/2023,2023-08-03,2023-06-26,BRABCBACNPR4,SUBSCRICAO,2023-06-30,,2023-12-20
1,BRABEVACNOR1,0.139941,16.090000,30/04/2014 a 22/05/2014,2014-05-29,2014-04-28,BRABEVACNOR1,SUBSCRICAO,2014-04-28,,2023-12-20
2,BRAESBACNOR7,0.538055,9.610000,09/12/2022 a 04/01/2023,2023-01-09,2022-12-05,BRAESBACNOR7,SUBSCRICAO,2022-12-08,,2023-12-20
3,BRAESOACNOR0,0.947563,1.637183,31/12/9999 a 05/12/2022,2022-12-08,2022-12-05,BRAESOACNOR0,SUBS C/ RENUNC,2022-12-06,,2023-12-20
4,BRAHEBACNOR0,14.097000,28.810000,18/02/2020 a 13/03/2020,2020-03-18,2020-02-17,BRAHEBACNOR0,SUBSCRICAO,2020-02-17,,2023-12-20


In [85]:
sd0_ = su0[["isinCode", "label", "lastDatePrior", "approvedOn", "subscriptionDate", "tradingPeriod", "percentage", "priceUnit"]]\
    .rename(columns={
        "isinCode": "isin",
        "label": "corporate_action_label",
        "lastDatePrior": "last_date_prior_ex",
        "approvedOn": "approved_on",
        "subscriptionDate": "subscription_date",
        "priceUnit": "price_unit",
        "tradingPeriod": "trading_period",
    })

sd0_["trading_period_start"] = pd.to_datetime(sd0_["trading_period"].str[:10], format="%d/%m/%Y", errors="coerce")
sd0_["trading_period_end"] = pd.to_datetime(sd0_["trading_period"].str[-10:], format="%d/%m/%Y", errors="coerce")
sd0_ = pd.merge(sd0_, symbols_properties[["isin", "symbol"]], on="isin").drop(columns="isin")
sd0_

,corporate_action_label,last_date_prior_ex,approved_on,subscription_date,trading_period,percentage,price_unit,trading_period_start,trading_period_end,symbol
0,SUBSCRICAO,2023-06-30,2023-06-26,2023-08-03,03/07/2023 a 31/07/2023,4.905471,14.485865,2023-07-03,2023-07-31,ABCB4
1,SUBSCRICAO,2014-04-28,2014-04-28,2014-05-29,30/04/2014 a 22/05/2014,0.139941,16.090000,2014-04-30,2014-05-22,ABEV3
2,SUBSCRICAO,2022-12-08,2022-12-05,2023-01-09,09/12/2022 a 04/01/2023,0.538055,9.610000,2022-12-09,2023-01-04,AESB3
3,SUBS C/ RENUNC,2022-12-06,2022-12-05,2022-12-08,31/12/9999 a 05/12/2022,0.947563,1.637183,NaT,2022-12-05,AESO3
4,SUBSCRICAO,2020-02-17,2020-02-17,2020-03-18,18/02/2020 a 13/03/2020,14.097000,28.810000,2020-02-18,2020-03-13,AHEB3
...,...,...,...,...,...,...,...,...,...,...
378,SUBSCRICAO,2021-01-12,2021-01-07,2021-02-11,13/01/2021 a 08/02/2021,20.759823,9.130000,2021-01-13,2021-02-08,VLID3
379,SUBSCRICAO,2021-08-09,2021-08-09,2021-09-10,10/08/2021 a 06/09/2021,4.818732,39.600000,2021-08-10,2021-09-06,VSPT3
380,SUBSCRICAO,2021-08-09,2021-08-09,2021-09-10,10/08/2021 a 06/09/2021,4.818732,39.600000,2021-08-10,2021-09-06,VSPT4
381,SUBSCRICAO,2013-11-05,2013-11-04,2013-12-05,06/11/2013 a 28/11/2013,120.000000,0.620000,2013-11-06,2013-11-28,VULC3


In [91]:
import pyarrow.compute as pc

brasa.get_dataset("b3-equities-register")\
    .filter(pc.field("refdate") == datetime(2023, 12, 13))\
    .filter(pc.field("instrument_market") == 10)\
    .scanner(columns=["symbol", "isin", "specification_code", "corporation_name", "cfi_code", "instrument_asset"])\
    .to_table()\
    .to_pandas()

,symbol,isin,specification_code,corporation_name,cfi_code,instrument_asset
0,CEAL11,BRCEALCDAM10,UNT MB,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,EMXXXR,CEAL
1,LOGG3,BRLOGGACNOR7,ON NM,LOG COMMERCIAL PROPERTIES,ESVUFR,LOGG
2,TSNC11,BRTSNCCTF009,CI,TRANSINC FDO INV IMOB - FII,CICIRU,TSNC
3,TECB11,BRTECBCTF004,CI,ÍNDICE DE AÇÕES TECH BRASIL ETF FUNDO DE ÍNDICE,CEOJEU,TECB
4,TTEN3,BRTTENACNOR0,ON NM,TRÊS TENTOS AGROINDUSTRIAL S/A,ESVUFR,TTEN
...,...,...,...,...,...,...
3327,BCEE5,BRBCEEACNPA6,PNA*,BANCO BESA S/A,EPNNPR,BCEE
3328,CMNS3,BRCMNSACNOR1,ON,CMN SOLUTIONS AO18 PARTICIPACOES S.A.,ESVUFR,CMNS
3329,ARMT11,BRARMTD01M10,DIR,ARCELORMITTAL,RSXXXX,ARMT
3330,TORD11,BRTORDCTF015,CI,TORDESILHAS EI FDO. INV. IMOB.,CICIRU,TORD


In [98]:
comp_det.head(3).transpose()

,0,1,2
issuingCompany,BBAS,BBAS,BBAS
companyName,BCO BRASIL S.A.,BCO BRASIL S.A.,BCO BRASIL S.A.
tradingName,BRASIL,BRASIL,BRASIL
cnpj,00000000000191,00000000000191,00000000000191
industryClassification,Financeiro / Intermediários Financeiros / Bancos,Financeiro / Intermediários Financeiros / Bancos,Financeiro / Intermediários Financeiros / Bancos
industryClassificationEng,None,None,None
activity,Banco Múltiplo,Banco Múltiplo,Banco Múltiplo
website,www.bb.com.br,www.bb.com.br,www.bb.com.br
hasQuotation,None,None,None
status,A,A,A


In [106]:
industry_sectors = comp_det["industryClassification"].str\
    .replace(r" +/ +", "/", regex=True).str\
    .split("/", expand=True)\
    .rename(columns={0: "sector", 1: "subsector", 2: "segment"})
industry_sectors["symbol"] = comp_det["code"]
industry_sectors

,sector,subsector,segment,symbol
0,Financeiro,Intermediários Financeiros,Bancos,BBAS11
1,Financeiro,Intermediários Financeiros,Bancos,BBAS12
2,Financeiro,Intermediários Financeiros,Bancos,BBAS3
3,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,ALPA3
4,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,ALPA4
...,...,...,...,...
607,Consumo Cíclico,Tecidos. Vestuário e Calçados,Fios e Tecidos,PTNT4
608,Financeiro,Intermediários Financeiros,Bancos,RPAD3
609,Financeiro,Intermediários Financeiros,Bancos,RPAD5
610,Financeiro,Intermediários Financeiros,Bancos,RPAD6


In [105]:
industry_sectors.query("segment == "Calçados"")

,sector,subsector,segment,symbol
3,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,ALPA3
4,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,ALPA4
31,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,VULC3
223,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,GRND3
283,Consumo Cíclico,Tecidos. Vestuário e Calçados,Calçados,CAMB3
